In [1]:
%run ../Python_files/util_data_storage_and_load.py

In [2]:
%run ../Python_files/load_dicts.py

In [3]:
%run ../Python_files/util.py

In [4]:
# load logit_route_choice_probability_matrix
P = zload('../temp_files/logit_route_choice_probability_matrix.pkz')

In [5]:
import numpy as np
from numpy.linalg import inv

P = np.matrix(P)

In [6]:
np.size(P, 0), np.size(P, 1)

(56, 140)

In [7]:
import numpy as np
from numpy.linalg import inv

def samp_cov(x):
    """
    x: sample matrix, each column is a link flow vector sample; 24 * K
    K: number of samples
    S: sample covariance matrix
    ----------------
    return: inv(S)
    ----------------
    """
    x = np.matrix(x)
    K = np.size(x, 1)
    x_mean = sum(x[:,k] for k in range(K)) / K
    S = sum(np.dot(x[:,k] - x_mean, np.transpose(x[:,k] - x_mean)) for k in range(K)) / (K - 1)
    return S

In [8]:
# load path-link incidence matrix
A = zload('../temp_files/path-link_incidence_matrix.pkz')

In [9]:
# load link counts data

import json

with open('../temp_files/link_day_minute_Jan_dict_JSON.json', 'r') as json_file:
    link_day_minute_Jan_dict_JSON = json.load(json_file)

In [10]:
week_day_Jan_list = [2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 23, 24, 25, 26, 27, 30, 31]

In [11]:
link_day_minute_Jan_list = []
for link_idx in range(24):
    for day in week_day_Jan_list: 
        for minute_idx in range(120):
            key = 'link_' + str(link_idx) + '_' + str(day)
            link_day_minute_Jan_list.append(link_day_minute_Jan_dict_JSON[key] ['PM_flow_minute'][minute_idx])

In [12]:
len(link_day_minute_Jan_list)

63360

In [13]:
x = np.matrix(link_day_minute_Jan_list)
x = np.matrix.reshape(x, 24, 2640)

x = np.nan_to_num(x)
y = np.array(np.transpose(x))
y = y[np.all(y != 0, axis=1)]
x = np.transpose(y)
x = np.matrix(x)

In [14]:
np.size(x, 0), np.size(x, 1)

(24, 2640)

In [15]:
x[:,:2]

matrix([[ 1230.12416328,  1279.38104524],
        [ 1228.05925487,  1209.42588982],
        [ 1536.42011653,  1541.94969038],
        [ 1482.21724808,  1493.13076857],
        [ 1563.84999771,  1546.95935776],
        [ 1421.64559003,  1417.08420916],
        [ 1218.21818385,  1265.1050915 ],
        [ 1313.79255409,  1286.83002547],
        [ 1652.3063151 ,  1697.08769505],
        [ 1680.54984494,  1671.7467482 ],
        [ 1342.67353554,  1430.76524469],
        [ 1269.89950211,  1271.68701548],
        [ 1247.66327751,  1247.22950748],
        [ 1327.46584828,  1347.45889852],
        [ 1244.98367115,  1304.71836289],
        [  942.74079074,   953.12973098],
        [ 1239.47556914,  1239.00705353],
        [ 1256.40520692,  1300.86700399],
        [ 1568.97186586,  1548.68494056],
        [ 1599.34311776,  1660.31884167],
        [ 1534.18834188,  1481.70426083],
        [ 1403.63055177,  1423.62591683],
        [ 1223.48363077,  1275.87847974],
        [ 1288.20548244,  1261.903

In [16]:
np.size(A,0), np.size(A,1)

(24, 140)

In [17]:
from gurobipy import *

L = 56  # dimension of lam

K = np.size(x, 1)
S = samp_cov(x)
inv_S = inv(S)

A_t = np.transpose(A)
P_t = np.transpose(P)
# PA'
PA_t = np.dot(P, A_t)
# AP_t
AP_t = np.transpose(PA_t)

Q = np.dot(np.dot(PA_t, inv_S), AP_t)
b = sum([np.dot(np.dot(PA_t, inv_S), x[:, k]) for k in range(K)])


model = Model("OD_matrix_estimation")

lam = []
for l in range(L):
    lam.append(model.addVar(name='lam_' + str(l)))

model.update() 

# Set objective: (K/2) lam' * Q * lam - b * lam
obj = 0
for i in range(L):
    for j in range(L):
        obj += (1.0 /2) * K * lam[i] * Q[i, j] * lam[j]
for l in range(L):
    obj += - b[l] * lam[l]
model.setObjective(obj)

# Add constraint: lam >= 0
for l in range(L):
    model.addConstr(lam[l] >= 0)

model.update() 

model.optimize()

lam_list = []
for v in model.getVars():
    print('%s %g' % (v.varName, v.x))
    lam_list.append(v.x)
print('Obj: %g' % obj.getValue())

Optimize a model with 56 rows, 56 columns and 56 nonzeros
Model has 1596 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [2e+01, 5e+03]
  Bounds range    [0e+00, 0e+00]
  RHS range       [0e+00, 0e+00]
Presolve removed 56 rows and 0 columns
Presolve time: 0.00s
Presolved: 0 rows, 56 columns, 0 nonzeros
Presolved model has 1596 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 25
 AA' NZ     : 3.000e+02
 Factor NZ  : 3.250e+02
 Factor Ops : 5.525e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -5.85460522e+07  0.00000000e+00  8.20e+03 1.32e+03  1.00e+06     0s
   1  -1.48862052e+07 -1.48137575e+07  6.08e+02 9.78e+01  1.76e+05     0s
   2  -1.16332007e+07 -1.24413900e+07  3.82e+01 6.15e+00  2.37e+04     0s
   3  -1.15339689e+07 -1.16950157e+07  3.82e-05 6.15e-06  2.8

In [18]:
# write estimation result to file
n = 8  # number of nodes
with open('../temp_files/OD_demand_matrix_Jan_weekday_PM.txt', 'w') as the_file:
    idx = 0
    for i in range(n + 1)[1:]:
        for j in range(n + 1)[1:]:
            if i != j: 
                the_file.write("%d,%d,%f\n" %(i, j, lam_list[idx]))
                idx += 1

In [19]:
lam_list

[6.7107862821140095e-12,
 1173.323701991738,
 7.366231796418669e-12,
 0.00021075033778821515,
 1522.617494558895,
 0.00035286075796152946,
 2.2228863049219443e-09,
 0.00013053277368749736,
 59.61129908386935,
 1351.4561866101872,
 0.0007429341065525417,
 0.00019901991161669938,
 0.0003528545990386465,
 0.00035043725392813274,
 706.1540472112325,
 0.023823167817572405,
 0.16791421058996092,
 1525.1327374934176,
 0.0001425546822462708,
 0.0003528859535953809,
 0.00035273500512810656,
 737.8605606300498,
 630.6490046789397,
 0.01678986861132259,
 0.21562578731195803,
 0.09397501314415757,
 1147.5220400171927,
 1219.2581772190192,
 9.48769533904601,
 0.016414023013980428,
 0.01467802659727857,
 1234.311658565556,
 1245.7767968161513,
 334.52867017694484,
 23.44533070142724,
 1.6707546223535228e-06,
 1.6706343705642498e-06,
 1.670758399036882e-06,
 0.39959800151921243,
 971.1204652173864,
 1036.3978495155689,
 538.5753855175842,
 1312.5451255876853,
 0.0234337968517013,
 5.180474015421396,
